### import stuff

In [1]:
import pandas as pd
from datetime import timedelta

### get subset of train data (because my kernel kept dying and its quicker

In [2]:
dtypes = {'id': 'uint32',
          'item_nbr': 'int32',
          'store_nbr': 'int8',
          'unit_sales': 'float32'}

In [3]:
train = pd.read_csv('./data/train.csv',
                    usecols=[1,2,3,4],
                    skiprows=range(1, 86672217),
                    parse_dates=['date'],
                    dtype=dtypes
                   )

In [4]:
train.head()

,date,store_nbr,item_nbr,unit_sales
0,2016-08-01,1,103520,3.0
1,2016-08-01,1,103665,2.0
2,2016-08-01,1,105574,7.0
3,2016-08-01,1,105575,13.0
4,2016-08-01,1,105577,2.0


### remove negative unit_sales

In [5]:
train.loc[(train.unit_sales<0), 'unit_sales'] = 0

In [6]:
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p)

### create column that can be used to identify day of week

In [7]:
train['dow'] = train['date'].dt.dayofweek

In [8]:
train.head()

,date,store_nbr,item_nbr,unit_sales,dow
0,2016-08-01,1,103520,1.386294,0
1,2016-08-01,1,103665,1.098612,0
2,2016-08-01,1,105574,2.079442,0
3,2016-08-01,1,105575,2.639057,0
4,2016-08-01,1,105577,1.098612,0


### create unique lists, restructure dataframe, remove nulls, restore dataframes original structure

In [9]:
u_dates = train.date.unique()

In [10]:
u_stores = train.store_nbr.unique()

In [11]:
u_items = train.item_nbr.unique()

In [12]:
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)

In [13]:
train.head()

unit_sales  dow
date       store_nbr item_nbr                 
2016-08-01 1         103520      1.386294    0
                     103665      1.098612    0
                     105574      2.079442    0
                     105575      2.639057    0
                     105577      1.098612    0

In [14]:
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date', 'store_nbr', 'item_nbr']
    )
)

In [15]:
train.head()

unit_sales  dow
date       store_nbr item_nbr                 
2016-08-01 1         103520      1.386294  0.0
                     103665      1.098612  0.0
                     105574      2.079442  0.0
                     105575      2.639057  0.0
                     105577      1.098612  0.0

### tell the gc he can come get these vectors (delete vectors)

In [16]:
del u_dates, u_stores, u_items

### fill NaNs with 0

In [17]:
train.loc[:, 'unit_sales'].fillna(0, inplace=True)

### remove multi-index

In [18]:
train.reset_index(inplace=True)

In [19]:
train.head()

,date,store_nbr,item_nbr,unit_sales,dow
0,2016-08-01,1,103520,1.386294,0.0
1,2016-08-01,1,103665,1.098612,0.0
2,2016-08-01,1,105574,2.079442,0.0
3,2016-08-01,1,105575,2.639057,0.0
4,2016-08-01,1,105577,1.098612,0.0


### Days of week means

In [20]:
lastdate = train.iloc[train.shape[0]-1].date

In [21]:
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')

In [22]:
ma_dw.head()

madw
item_nbr store_nbr dow          
96995    1         0.0  0.895880
                   1.0  0.693147
                   2.0  0.693147
                   3.0  0.794513
                   4.0  0.943827

In [23]:
ma_dw.reset_index(inplace=True)

In [24]:
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')

In [25]:
ma_wk.reset_index(inplace=True)

In [26]:
ma_wk.head()

,store_nbr,item_nbr,mawk
0,1,96995,0.844024
1,1,99197,1.010969
2,1,103520,1.114774
3,1,103665,1.320085
4,1,105574,1.693974


### Moving averages

In [27]:
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')

In [28]:
ma_is.head()

mais226
item_nbr store_nbr          
96995    1          0.056869
         2          0.069360
         3          0.096034
         4          0.027744
         5          0.040857

In [29]:
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

In [30]:
del tmp,tmpg,train

In [31]:
ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)

In [32]:
#Load test
test = pd.read_csv('./data/test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

In [33]:
del ma_is, ma_wk, ma_dw

In [34]:
#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1)

In [35]:
#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

In [36]:
test[['id','unit_sales']].to_csv('jhill.csv.gz', index=False, float_format='%.3f', compression='gzip')